# Nerfstudio Training: NeRF vs 3D Gaussian Splatting

This notebook trains both methods using Nerfstudio for fair comparison.

**Methods:**
- NeRF: `nerfacto` (fast NeRF variant)
- 3DGS: `splatfacto` (Gaussian Splatting with gsplat)

**Expected Time:** ~5-10 minutes each

## Step 1: Check GPU

In [5]:
!nvidia-smi

import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Wed Nov 19 15:07:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Step 2: Install Nerfstudio

In [6]:
# Install nerfstudio and dependencies
!pip install nerfstudio -q

print("\n✓ Nerfstudio installed")

# Verify installation
!ns-train --help | head -20


✓ Nerfstudio installed
2025-11-19 15:07:43.229471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763564863.249398    2678 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763564863.255447    2678 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763564863.271265    2678 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763564863.271288    2678 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763564863.271292    2678 computation_placer.cc:177

## Step 3: Setup Working Directory

In [7]:
import os

# Create working directory
%cd /content
!mkdir -p nerfstudio_training
%cd nerfstudio_training

print("\n✓ Working directory created")
!pwd

/content
/content/nerfstudio_training

✓ Working directory created
/content/nerfstudio_training


## Step 4: Mount Drive & Copy Dataset

In [8]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✓ Drive mounted")

Mounted at /content/drive

✓ Drive mounted


In [9]:
# Copy dataset
!mkdir -p data/nerf_synthetic
!cp -r /content/drive/MyDrive/data/nerf_synthetic/lego data/nerf_synthetic/

print("✓ Dataset copied")

# Verify
!ls -lh data/nerf_synthetic/lego/

✓ Dataset copied
total 376K
drwx------ 2 root root  20K Nov 19 15:08 test
drwx------ 2 root root 4.0K Nov 19 15:08 train
-rw------- 1 root root 172K Nov 19 15:08 transforms_test.json
-rw------- 1 root root  86K Nov 19 15:08 transforms_train.json
-rw------- 1 root root  86K Nov 19 15:08 transforms_val.json
drwx------ 2 root root 4.0K Nov 19 15:08 val


## Step 5: Convert Dataset for Nerfstudio

Nerfstudio needs the data in a specific format

In [10]:
# Nerfstudio can read blender format directly!
# Just need to verify the structure

import json

print("Checking dataset format...\n")

for split in ['train', 'val', 'test']:
    json_path = f'data/nerf_synthetic/lego/transforms_{split}.json'
    if os.path.exists(json_path):
        with open(json_path) as f:
            data = json.load(f)
        print(f"✓ {split}: {len(data['frames'])} frames")
    else:
        print(f"✗ {split}: not found")

print("\n✓ Dataset ready for Nerfstudio")

Checking dataset format...

✓ train: 100 frames
✓ val: 100 frames
✓ test: 200 frames

✓ Dataset ready for Nerfstudio


## Step 6: Train NeRF (Nerfacto)

**This will take ~5-10 minutes**

In [15]:
# Train NeRF using nerfacto method
!ns-train nerfacto \
    --data data/nerf_synthetic/lego \
    --pipeline.model.camera-optimizer.mode off \
    --max-num-iterations 10000 \
    --viewer.quit-on-train-completion True \
    blender-data

Streaming output truncated to the last 5000 lines.
6400 (64.00%)       270.398 ms           16 m, 13 s           15.51 K                                
6410 (64.10%)       278.646 ms           16 m, 40 s           15.16 K                                
6420 (64.20%)       276.643 ms           16 m, 30 s           15.18 K                                
6430 (64.30%)       275.196 ms           16 m, 22 s           15.23 K                                
6440 (64.40%)       277.481 ms           16 m, 27 s           15.21 K                                
---------------------------------------------------------------------------------------------------- 
Viewer running locally at: http://localhost:7007 (listening on 0.0.0.0)                              
Step (% Done)       Train Iter (time)    ETA (time)           Train Rays / Sec                       
-----------------------------------------------------------------------------------                  
6360 (63.60%)       270.968 ms 

## Step 7: Evaluate NeRF Results

In [12]:
# Find the latest nerfacto config
!ls -lht outputs/lego/nerfacto/*/config.yml | head -1

# Store the path (you'll need to update this with actual path)
nerf_config = !ls outputs/lego/nerfacto/*/config.yml | head -1
nerf_config = nerf_config[0]
print(f"\nNeRF config: {nerf_config}")

ls: cannot access 'outputs/lego/nerfacto/*/config.yml': No such file or directory

NeRF config: ls: cannot access 'outputs/lego/nerfacto/*/config.yml': No such file or directory


In [17]:
# Find the actual config path
import glob

# Check what's in outputs
!ls -la outputs/

# Find the config
configs = glob.glob('outputs/lego/nerfacto/*/config.yml')
if configs:
    nerf_config = configs[0]
    print(f"Found config: {nerf_config}")
else:
    # Try without the lego subdirectory
    configs = glob.glob('outputs/*/nerfacto/*/config.yml')
    if configs:
        nerf_config = configs[0]
        print(f"Found config: {nerf_config}")
    else:
        print("Config not found, checking all outputs:")
        !find outputs -name "config.yml" -type f

total 12
drwxr-xr-x 3 root root 4096 Nov 19 15:11 .
drwxr-xr-x 4 root root 4096 Nov 19 15:11 ..
drwxr-xr-x 3 root root 4096 Nov 19 15:11 lego
Found config: outputs/lego/nerfacto/2025-11-19_151135/config.yml


In [19]:
import json
import os

# Fix the JSON files for nerfstudio
for split in ['train', 'val', 'test']:
    json_path = f'data/nerf_synthetic/lego/transforms_{split}.json'

    with open(json_path, 'r') as f:
        data = json.load(f)

    # Get actual files (excluding depth maps)
    split_dir = f'data/nerf_synthetic/lego/{split}'
    actual_files = sorted([f.replace('.png', '') for f in os.listdir(split_dir)
                          if f.endswith('.png') and 'depth' not in f])

    print(f"{split}: Found {len(actual_files)} images")
    print(f"  First few: {actual_files[:3]}")

    # Update frame paths
    for i, frame in enumerate(data['frames']):
        if i < len(actual_files):
            data['frames'][i]['file_path'] = f"./{split}/{actual_files[i]}"

    # Save fixed JSON
    with open(json_path, 'w') as f:
        json.dump(data, f, indent=2)

    print(f"  ✓ Fixed {split}")

print("\n✓ All JSON files fixed!")

train: Found 100 images
  First few: ['r_0', 'r_1', 'r_10']
  ✓ Fixed train
val: Found 100 images
  First few: ['r_0', 'r_1', 'r_10']
  ✓ Fixed val
test: Found 196 images
  First few: ['r_101', 'r_102', 'r_103']
  ✓ Fixed test

✓ All JSON files fixed!


In [ ]:
# Patch nerfstudio's eval_utils.py to disable weights_only
import fileinput
import sys

eval_utils_path = '/usr/local/lib/python3.12/dist-packages/nerfstudio/utils/eval_utils.py'

# Read the file
with open(eval_utils_path, 'r') as f:
    content = f.read()

# Replace the torch.load line
content = content.replace(
    'loaded_state = torch.load(load_path, map_location="cpu")',
    'loaded_state = torch.load(load_path, map_location="cpu", weights_only=False)'
)

# Write it back
with open(eval_utils_path, 'w') as f:
    f.write(content)

print("✓ Patched eval_utils.py")

In [4]:
# Search entire system for any nerfacto outputs
!find /content -name "*nerfacto*" 2>/dev/null
!find /content -name "config.yml" 2>/dev/null
!find /content -name "*2025-11-19*" 2>/dev/null

# Check what's actually in /content
!ls -la /content/

# Check if anything is in Google Drive
!ls -la /content/drive/MyDrive/ 2>/dev/null | head -20

total 16
drwxr-xr-x 1 root root 4096 Nov 17 14:29 .
drwxr-xr-x 1 root root 4096 Nov 19 19:40 ..
drwxr-xr-x 4 root root 4096 Nov 17 14:29 .config
drwxr-xr-x 1 root root 4096 Nov 17 14:29 sample_data


In [21]:
# Quick fix: Use older PyTorch behavior
import torch
torch.serialization.weights_only = False

# Set the config path
nerf_config = "outputs/lego/nerfacto/2025-11-19_151135/config.yml"
print(f"NeRF config: {nerf_config}")

# Now evaluate
!ns-eval \
    --load-config {nerf_config} \
    --output-path nerf_eval_results.json

NeRF config: outputs/lego/nerfacto/2025-11-19_151135/config.yml
2025-11-19 16:17:55.183428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763569075.208809   22359 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763569075.215055   22359 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763569075.230987   22359 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763569075.231012   22359 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763569075.

In [14]:
# View results
import json

with open('nerf_eval_results.json', 'r') as f:
    nerf_results = json.load(f)

print("="*60)
print("NERF (NERFACTO) RESULTS")
print("="*60)
print(json.dumps(nerf_results, indent=2))
print("="*60)

FileNotFoundError: [Errno 2] No such file or directory: 'nerf_eval_results.json'

## Step 8: Train 3D Gaussian Splatting (Splatfacto)

**This will take ~5-10 minutes**

In [ ]:
# Train 3DGS using splatfacto method
!ns-train splatfacto \
    --data data/nerf_synthetic/lego \
    --pipeline.datamanager.camera-optimizer.mode off \
    --max-num-iterations 10000 \
    --viewer.quit-on-train-completion True \
    blender-data

## Step 9: Evaluate 3DGS Results

In [ ]:
# Find the latest splatfacto config
!ls -lht outputs/lego/splatfacto/*/config.yml | head -1

# Store the path
gs_config = !ls outputs/lego/splatfacto/*/config.yml | head -1
gs_config = gs_config[0]
print(f"\n3DGS config: {gs_config}")

In [ ]:
# Evaluate on test set
!ns-eval \
    --load-config {gs_config} \
    --output-path gs_eval_results.json

In [ ]:
# View results
with open('gs_eval_results.json', 'r') as f:
    gs_results = json.load(f)

print("="*60)
print("3D GAUSSIAN SPLATTING (SPLATFACTO) RESULTS")
print("="*60)
print(json.dumps(gs_results, indent=2))
print("="*60)

## Step 10: Side-by-Side Comparison

In [ ]:
import pandas as pd

# Create comparison table
comparison = {
    'Metric': [],
    'NeRF (Nerfacto)': [],
    '3DGS (Splatfacto)': []
}

# Extract metrics
for key in ['psnr', 'ssim', 'lpips']:
    if key in nerf_results['results']:
        comparison['Metric'].append(key.upper())
        comparison['NeRF (Nerfacto)'].append(f"{nerf_results['results'][key]:.4f}")
        comparison['3DGS (Splatfacto)'].append(f"{gs_results['results'][key]:.4f}")

df = pd.DataFrame(comparison)

print("\n" + "="*60)
print("COMPARISON: NeRF vs 3D Gaussian Splatting")
print("="*60)
print(df.to_string(index=False))
print("="*60)

# Save to CSV
df.to_csv('comparison_results.csv', index=False)
print("\n✓ Saved to comparison_results.csv")

## Step 11: Render Test Images

In [ ]:
# Render NeRF test images
!ns-render camera-path \
    --load-config {nerf_config} \
    --camera-path-filename data/nerf_synthetic/lego/transforms_test.json \
    --output-path renders/nerf/

print("\n✓ NeRF images rendered to renders/nerf/")

In [ ]:
# Render 3DGS test images
!ns-render camera-path \
    --load-config {gs_config} \
    --camera-path-filename data/nerf_synthetic/lego/transforms_test.json \
    --output-path renders/3dgs/

print("\n✓ 3DGS images rendered to renders/3dgs/")

## Step 12: View Sample Images

In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt

# Show first rendered image from each method
nerf_imgs = !ls renders/nerf/*.png | head -3
gs_imgs = !ls renders/3dgs/*.png | head -3

print("Sample Renderings:\n")

for i in range(min(3, len(nerf_imgs))):
    print(f"\n--- Image {i+1} ---")
    print("NeRF:")
    display(Image(nerf_imgs[i], width=300))
    print("3DGS:")
    display(Image(gs_imgs[i], width=300))

## Step 13: Download Results

In [ ]:
from google.colab import files

# Zip everything
!zip -r nerfstudio_results.zip \
    nerf_eval_results.json \
    gs_eval_results.json \
    comparison_results.csv \
    renders/

print("\n✓ Results zipped")

# Download
files.download('nerfstudio_results.zip')

print("\n✓ Download started")

## Step 14: Summary for Report

In [ ]:
print("="*60)
print("SUMMARY FOR YOUR REPORT")
print("="*60)
print(f"\nNeRF (Nerfacto):")
print(f"  PSNR: {nerf_results['results']['psnr']:.2f} dB")
print(f"  SSIM: {nerf_results['results']['ssim']:.4f}")
print(f"  LPIPS: {nerf_results['results']['lpips']:.4f}")

print(f"\n3D Gaussian Splatting (Splatfacto):")
print(f"  PSNR: {gs_results['results']['psnr']:.2f} dB")
print(f"  SSIM: {gs_results['results']['ssim']:.4f}")
print(f"  LPIPS: {gs_results['results']['lpips']:.4f}")

# Calculate differences
psnr_diff = gs_results['results']['psnr'] - nerf_results['results']['psnr']
ssim_diff = gs_results['results']['ssim'] - nerf_results['results']['ssim']

print(f"\nDifference (3DGS - NeRF):")
print(f"  PSNR: {psnr_diff:+.2f} dB ({'+' if psnr_diff > 0 else ''}{(psnr_diff/nerf_results['results']['psnr']*100):.1f}%)")
print(f"  SSIM: {ssim_diff:+.4f} ({'+' if ssim_diff > 0 else ''}{(ssim_diff/nerf_results['results']['ssim']*100):.1f}%)")

print("\n" + "="*60)
print("\nBoth methods trained for 10,000 iterations using Nerfstudio.")
print("Dataset: NeRF Synthetic LEGO (Blender)")
print("="*60)

---

## Notes:

**Advantages of this approach:**
- ✅ Same framework (Nerfstudio) for both methods
- ✅ Fair comparison with equal iterations
- ✅ Much faster than vanilla implementations
- ✅ Automatic metrics calculation
- ✅ Professional rendered outputs

**For your report:**
- Both trained for 10k iterations
- Same dataset and hardware
- Apples-to-apples comparison
- Can cite Nerfstudio framework